Livedoorニュースコーパスを使った実験

http://www.rondhuit.com/download.html#ldcc

In [1]:
from os import listdir
from os.path import join, normpath, isdir
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

## Load data

In [2]:
rx_filename = re.compile('^.*\d+\.txt$')

In [3]:
texts = []
labels = np.array([], dtype=np.uint8)
label_names = []

datadir = normpath('../data/livedoornews')
subdirs = listdir(datadir)
label = 0
for subdir in subdirs:
    subdir_path = join(datadir, subdir)
    if not isdir(subdir_path):
        continue
    
    texts_of_this_label = []
    textfiles = listdir(subdir_path)
    for j, textfile in enumerate(textfiles):
        if not rx_filename.match(textfile):
            continue
            
        textfile_path = join(subdir_path, textfile)
        
        with open(textfile_path, 'r') as f:
            texts_of_this_label.append(f.read())
            
    
    texts.extend(texts_of_this_label)
    labels = np.hstack((labels, np.ones(len(texts_of_this_label), dtype=np.uint8) * label))
    label_names.append(subdir)
    
    label += 1
    print subdir

dokujo-tsushin
it-life-hack
kaden-channel
livedoor-homme
movie-enter
peachy
smax
sports-watch
topic-news


## Define tokenizer

In [4]:
import MeCab
import neologdn
import re

class MeCabTokenizer(object):
    def __init__(self, dicpath=None):
        if dicpath is not None:
            dicpath = ' -d ' + dicpath
        else:
            dicpath = ''
            
        self.tagger = MeCab.Tagger('-Ochasen' + dicpath)
        self.rx_url = re.compile('https?://[\w/:%#\$&\?\(\)~\.=\+\-]+')
    
    def __call__(self, text):
        return self.tokenize(text)
    
    def normalize_word(self, text):
        text = self.rx_url.sub('URLTOKEN', text)
        return text

    def tokenize(self, text):
        input_type = type(text)
        bow = []
        
        #text = neologdn.normalize(text.decode('utf8')).encode('utf8')
        if input_type == str:
            text = text.decode('utf8')
        
        text = neologdn.normalize(text).encode('utf8')
        text = self.normalize_word(text)
        
        node = self.tagger.parseToNode(text)
        while node:
            feature = node.feature.split(',')
            
            if feature[0] == 'BOS/EOS':
                node = node.next
                continue
            
            if feature[0] in ['名詞', '動詞', '形容詞', '副詞', '連体詞']:
                word = feature[6] if feature[6] != '*' else node.surface
                
                if feature[1] == '数':
                    word = 'NUMTOKEN'
                
                if feature[1] == '接尾':
                    node = node.next
                    continue
                
                if input_type == unicode:
                    word = word.decode('utf8')
                
                bow.append(word)
                
            node = node.next
        
        return bow

In [5]:
tokenizer = MeCabTokenizer('/usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
for w in tokenizer('今日は焼肉が１０食食べたいな'):
    print w

今日
焼肉
NUMTOKEN
食べる


## Train and test

In [6]:
from sklearn.ensemble import RandomForestClassifier

np.random.seed(42)
indices = np.random.permutation(len(texts))
X = pd.Series(texts)[indices]
Y = labels[indices]

n_test = 500
X_test = X[:n_test]
X_train = X[n_test:]
Y_test = Y[:n_test]
Y_train = Y[n_test:]

tokenizer = MeCabTokenizer()
vectorizer = TfidfVectorizer(analyzer=tokenizer)
classifier = RandomForestClassifier(n_estimators=120)

pipeline = Pipeline([
        ('tfidf', vectorizer),
        ('classifier', classifier),
    ])
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer=<__main__.MeCabTokenizer object at 0x139bfee90>,
        binary=False, decode_error=u'strict', dtype=<type 'numpy.int64'>,
        encoding=u'utf-8', input=u'content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), norm=...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [7]:
from sklearn.metrics import accuracy_score
Y_pred = pipeline.predict(X_test)
print accuracy_score(Y_pred, Y_test)

0.936


In [8]:
print len(vectorizer.get_feature_names())

63515


## 定性評価

In [9]:
test_input = """
アップルは米国で発表会を行い同社のスマートフォンの新モデルとなる「iPhone 7」および、大型サイズの「iPhone 5 plus」を発表した。

画面サイズはiPhone 7が4.7インチ、iPhone 7 Plusが5.5インチと前モデルから変わっていない。カラバリとして新色として「ジェットブラック」と呼ばれるテカテカ系の黒（いわゆるピアノブラック）、これにテカテカしてないつや消しブラックの「ブラック」、黒だけで2色が追加され全部で5色のバリエーションとなっている。

■予約開始は明日から！
予約開始日は9月9日からとなっている。出荷は9月16日。価格はiPhone 7が7万2,800円（税別）、iPhone 7 Plusが8万5,800円（同）からとなっている。

今回の最大の目玉となるのがFeliCa対応だ。おサイフケータイ非対応のためiPhone購入をあきらめていた人たちにとって新規導入の追い風になるのは間違いない。これまでiPhoneシリーズを一度も購入したことのない筆者でも、FeliCa対応なら買ってもいいかなと思ったくらい。

■防水・防塵性能に高機能カメラ
大画面モデルとなるiPhone 7 Plusには、予想されていた通り2レンズ＆センサー仕様のメインカメラが搭載されてきた。

iPhone 7はシングルレンズ仕様ながらも明るさがF1.8となる明るいレンズでストロボの使用を極力防いでくれるようになった。画素数は従来と同じながら、ノイズを極力減らすDTI（Deep Trench Isolation）構造採用によって画質の向上を図っている。光学手ブレ補正機能が搭載されたのもうれしい。

事前の噂通り本体が防水・防じんとなり、スピーカーがステレオ化し、3.5mmヘッドフォンジャックが廃止されているのも確認できた。水没iPhoneの悲劇は、これまで以上に発生しにくくなるというわけだ。

■新型画像エンジンにより処理も高速化
画像処理エンジンが新型になり高速化している点も見逃せない。実にスループットは従来比2倍を実現。オートフォーカスが高速化されたことで、シャッターチャンスを逃さない。ホワイトバランスの精度の向上によってよりリアリティの高い発色を実現している。LEDフラッシュ（ライト）は4灯となり、1.5倍明るくなっている。

日本でiPhoneの売れ行きが落ちていると言われていたところに今回の7および7 Plus登場。FeliCa対応などは、明らかに日本市場にターゲットを絞った新機能であり、アップルが日本市場を重視し、ユーザーニーズを押さえにきていることがわかると言えるだろう。
"""

y_pred_proba = pipeline.predict_proba([test_input])[0]
y_pred = np.argmax(y_pred_proba)
print 'Predicted label: ', label_names[y_pred]
print '============='
for label_name, proba in zip(label_names, y_pred_proba):
    print label_name, proba

Predicted label:  it-life-hack
dokujo-tsushin 0.0833333333333
it-life-hack 0.325
kaden-channel 0.2
livedoor-homme 0.133333333333
movie-enter 0.0166666666667
peachy 0.0916666666667
smax 0.116666666667
sports-watch 0.0333333333333
topic-news 0.0
